In [1]:
import os
import sys
import glob
import json
import numpy as np
import pandas as pd


In [2]:
"""
Quick script to convert network architecture config files into
a single, human-readable table of model architectures.
"""

def get_layer(dfi, n_classes=[]):
    """
    """
    name = dfi.layer_name.lower()
    num = "".join(_ for _ in name if _.isdigit())
    if "flatten" in name:
        layer = "flatten"
    elif "reshape" in name:
        layer = None
    elif "dropout" in name:
        layer = f"dropout"
    elif "fc" in name:
        if "intermediate" in name:
            num = "0"
        else:
            num = ""
        layer = f"fc{num}"
        units = dfi.args.get("units", n_classes)
        if not isinstance(units, list):
            units = [units]
        layer += " {}".format(units)
        if "norm" in name:
            layer = f"norm_fc{num}"
        if "relu" in name:
            layer = f"relu_fc{num}"
    elif "conv" in name:
        layer = f"conv{num}"
        layer += " {}".format(dfi.args["kernel_size"] + [dfi.args.get("units", dfi.args["filters"])])
    elif "pool" in name:
        layer = f"pool{num}"
        if dfi.args.get("strides", None) is not None:
            layer += " {}".format(dfi.args["strides"])
        else:
            layer += " {}".format(dfi.args["pool_size"])
        if "max" in dfi.layer_type.lower():
            layer = "m" + layer 
        else:
            layer = "h" + layer 
    elif "relu" in name:
        layer = f"relu{num}"
    elif "norm" in name:
        layer = f"norm{num}"
        if "batch" in dfi.layer_type.lower():
            layer = "b" + layer 
        elif "layer" in dfi.layer_type.lower():
            layer = "l" + layer 
        if "input" in name:
            layer = "input_" + layer
    else:
        layer = None
    return layer


list_regex_fn_arch = [
    "models/sound_localization/IHC3000/arch*/arch.json",
    "models/spkr_word_recognition/IHC3000/arch*/arch.json",
    "../freqdiscriminet/models/freq_quarter_octave/0250hz/IHC3000/arch*/arch.json",
]
list_df = []
for regex_fn_arch in list_regex_fn_arch:
    list_fn_arch = glob.glob(regex_fn_arch)
    for itr_arch, fn_arch in enumerate(sorted(list_fn_arch)):
        tag_arch = os.path.basename(os.path.dirname(fn_arch))
        if "arch_0346_00" in tag_arch:
            tag_arch = tag_arch.replace("arch_0346_00", "arch_f")
        df = pd.read_json(fn_arch)
        df["tag_arch"] = tag_arch
        df["itr_arch"] = itr_arch
        fn_config = fn_arch.replace("arch.json", "config.json")
        with open(fn_config, "r") as f:
            CONFIG = json.load(f)
            n_classes = [v for k, v in CONFIG["n_classes_dict"].items()]
        df["layer_name"] = df["args"].map(lambda _: _["name"])
        df["layer"] = df.apply(lambda _: get_layer(_, n_classes=n_classes), axis=1)
        df = df[~df["layer"].isna()].reset_index(drop=True)
        df = df.drop(columns=["args", "layer_type", "layer_name"])
        list_df.append(df)
df = pd.concat(list_df)
table_df = {}
for name, sub_df in df.groupby("tag_arch"):
    table_df[name] = sub_df["layer"]
table_df = pd.DataFrame(table_df).fillna("")
table_df.to_csv("data/misc/model_architecture_table.csv", index=False)
table_df


,arch01,arch02,arch03,arch04,arch05,arch06,arch07,arch08,arch09,arch0_0000,...,arch_f00,arch_f04,arch_f05,arch_f06,arch_f09,arch_f11,arch_f13,arch_f15,arch_f17,arch_f20
0,"conv0 [1, 8, 32]","conv0 [2, 8, 32]","conv0 [1, 4, 32]","conv0 [3, 8, 32]","conv0 [2, 32, 32]","conv0 [1, 64, 32]","conv0 [1, 16, 32]","conv0 [1, 64, 32]","conv0 [3, 32, 32]",input_lnorm,...,"conv0 [3, 53, 32]","conv0 [3, 53, 64]","conv0 [3, 53, 32]","conv0 [3, 53, 64]","conv0 [3, 53, 64]","conv0 [3, 53, 32]","conv0 [3, 53, 32]","conv0 [3, 53, 64]","conv0 [3, 53, 64]","conv0 [3, 53, 64]"
1,"mpool0 [1, 1]","mpool0 [1, 1]","mpool0 [1, 1]","mpool0 [1, 1]","mpool0 [1, 2]","mpool0 [1, 8]","mpool0 [1, 1]","mpool0 [1, 1]","mpool0 [1, 1]","conv0 [2, 42, 32]",...,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0
2,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,relu0,...,"hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]","hpool0 [1, 2]"
3,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,"hpool0 [2, 4]",...,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0,bnorm0
4,"conv1 [1, 64, 32]","conv1 [3, 16, 32]","conv1 [3, 32, 32]","conv1 [3, 8, 32]","conv1 [1, 4, 64]","conv1 [2, 4, 64]","conv1 [1, 8, 32]","conv1 [2, 16, 32]","conv1 [2, 16, 32]",lnorm0,...,"conv1 [1, 60, 64]","conv1 [1, 60, 64]","conv1 [1, 60, 128]","conv1 [1, 60, 128]","conv1 [3, 20, 128]","conv1 [1, 60, 64]","conv1 [3, 20, 64]","conv1 [1, 60, 64]","conv1 [1, 60, 128]","conv1 [3, 20, 128]"
5,"mpool1 [1, 1]","mpool1 [1, 1]","mpool1 [1, 8]","mpool1 [1, 2]","mpool1 [1, 4]","mpool1 [1, 1]","mpool1 [1, 2]","mpool1 [1, 8]","mpool1 [1, 4]","conv1 [2, 18, 64]",...,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1
6,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,relu1,...,"hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]","hpool1 [2, 4]"
7,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,"hpool1 [2, 4]",...,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1,bnorm1
8,"conv2 [1, 64, 32]","conv2 [2, 4, 32]","conv2 [3, 32, 64]","conv2 [1, 32, 64]","conv2 [3, 2, 64]","conv2 [1, 32, 64]","conv2 [2, 4, 64]","conv2 [2, 4, 64]","conv2 [2, 32, 64]",lnorm1,...,"conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]","conv2 [3, 46, 128]"
9,"mpool2 [1, 8]","mpool2 [1, 8]","mpool2 [1, 1]","mpool2 [1, 1]","mpool2 [1, 1]","mpool2 [2, 4]","mpool2 [1, 1]","mpool2 [1, 1]","mpool2 [1, 1]","conv2 [6, 6, 128]",...,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2,relu2
